In [45]:
# 导入库
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import MultiLabelBinarizer

In [37]:
# 读取清洗后的数据
df = pd.read_json("../data/cleaned_data.json", orient="records", encoding="utf-8")

In [51]:
# 查看数据
df.head(5)

,title,douban_score,watch_time,user_score,year,region,genre,director,cast
0,爱，死亡和机器人 第四季,4.9,2025-05-16 13:26:29,3,2025.0,[美国],"[剧情, 喜剧, 动作, 动画, 恐怖, 奇幻, 冒险]","[大卫·芬奇, 罗伯特·比斯, 安迪·里昂, 吕寅荣, 罗伯特·瓦利, 帕特里克·奥斯本, ...","[红辣椒乐队, 安东尼·凯迪斯]"
1,爱，死亡和机器人 第二季,6.9,2025-05-15 23:24:54,4,2021.0,[美国],"[喜剧, 科幻, 动画, 恐怖, 短片, 奇幻]","[蒂姆·米勒, 肉食部门, 罗伯特·瓦利, 吕寅荣, 里昂·贝雷尔, 多米尼克·博伊丁, 雷...","[诺兰·诺斯, 艾米丽·奥布莱恩]"
2,爱，死亡和机器人 第三季,8.5,2025-05-15 23:24:24,5,2022.0,[美国],"[喜剧, 科幻, 动画, 恐怖, 短片, 奇幻]","[帕特里克·奥斯本, 大卫·芬奇, 埃米莉·迪恩, 罗伯特·比斯, 安迪·里昂, 吕寅荣, ...","[乔什·布雷纳, 加里·安东尼·威廉斯]"
3,马勒冈的超级男孩,NaN,2025-05-11 19:22:01,4,2024.0,[印度],"[剧情, 喜剧]",[里马·卡蒂],"[阿达什·古拉夫, Manjiri, Pupala]"
4,惊天魔盗团,7.8,2025-05-03 12:54:07,4,2013.0,"[美国, 法国]","[剧情, 悬疑, 犯罪]",[路易斯·莱特里尔],"[杰西·艾森伯格, 艾拉·菲舍尔]"


In [61]:
# 进行 One-Hot 编码
# 需要编码的列
multilabel_columns = ["region", "genre", "director", "cast"]

# 定义一个函数来安全地处理列表，如果不是列表则尝试转换，否则返回空列表
def safe_list_conversion(value):
    if isinstance(value, list):
        return value
    elif isinstance(value, str):
        # 尝试按逗号分割字符串，并去除首尾空白
        return [item.strip() for item in value.split(',')]
    elif pd.isna(value):  # 显式检查是否为 NaN
        return []
    else:
        return []

# 对需要进行多标签编码的列应用转换函数
for col in multilabel_columns:
    df[col] = df[col].apply(safe_list_conversion)

# 进行 One-Hot 编码
encoded_parts = []
for col in multilabel_columns:
    mlb = MultiLabelBinarizer()
    dummies = pd.DataFrame(mlb.fit_transform(df[col]),
                           columns=[f"{col}_{cls}" for cls in mlb.classes_],
                           index=df.index)
    encoded_parts.append(dummies)

# 合并原始数据（去除多标签列）和编码结果
df_encoded = pd.concat([df.drop(columns=multilabel_columns)] + encoded_parts, axis=1)

# 保存结果
with open("../data/onehot_encoded_data.json", "w", encoding="utf-8") as f:
    df_encoded.to_json(f, orient="records", force_ascii=False, indent=2)

print("✅ One-Hot 编码完成，文件保存为 ../data/onehot_encoded_data.json")

✅ One-Hot 编码完成，文件保存为 ../data/onehot_encoded_data.json
